# Tutorial: Loading an USBMD data file
In this tutorial notebook we will show how to load an USBMD data file and how to access the data stored in it.

In [ ]:
# Set the working directory to the root of the repository
# We do this by moving up until we find the file users.yaml (which is in the
# root of the repository)
import os

while not os.path.exists('users.yaml'):
    os.chdir('..')

## Viewing the file structure
The USBMD data format works with HDF5 files. We can open a USBMD data file using the `h5py` package and have a look at the contents using the print_hdf5_attrs function. You can see that every dataset element contains a corresponding description and unit.

We will demonstrate this by loading some example cardiac data from the NAS. We assume that the `Ultrasound-BMD` folder on the NAS is mapped to `Z:/` on your computer.

> *Tip:*
> You can also use the [HDFView](https://www.hdfgroup.org/downloads/hdfview/) tool to view the contents of the USBMD data file without having to run any code.

In [ ]:
import warnings
import h5py
from usbmd.utils.utils import print_hdf5_attrs
import cv2

# Define path to the data file
data_path = r"Z:\Ultrasfdound-BMd\data\USBMD-example-data\planewave_l115v.hdf5"

try:
    # Open the file and print the contents
    with h5py.File(data_path, 'r') as f:
        print_hdf5_attrs(f)


except FileNotFoundError:
    from usbmd.data_format.usbmd_data_format import generate_example_dataset
    data_path = r"generated_dummy_data.hdf5"
    generate_example_dataset(data_path, add_optional_fields=True)
    warnings.warn(f"Data file not found. Generated dummy data file.")

## Loading the file with the toolbox

In [ ]:
from usbmd.data_format.usbmd_data_format import load_usbmd_file
import numpy as np


# Load the data file and construct a probe and scan object
# We will only load the first two frames of the data file
data, scan, probe = load_usbmd_file(data_path, frames=[0, 1])

# Print some info about the data
print('Data file loaded successfully')
print('The data tensor has shape: {}'.format(data.shape))
print('The dimensions of the data are (n_frames, n_transmits, n_elements, '
      'n_axial_samples, n_rf_iq_channels)')

## Beamforming the data
Now we would like to beamform the data to generate an image. For this we need to load a config file that contains the beamforming parameters. We can load a config using the `load_config_from_yaml` function. Once we have a config we create the beamformer using get_beamformer with the scan and probe class that we got from the data file.

In [ ]:
from pathlib import Path
from usbmd.pytorch_ultrasound.layers.beamformers import get_beamformer
from usbmd.utils.config import load_config_from_yaml
from usbmd.utils.config_validation import check_config

# Load the config file
config_path = Path('configs', 'config_usbmd_iq.yaml')
config = load_config_from_yaml(config_path)

# Check the config file for errors
check_config(config)

# Create the beamformer
beamformer = get_beamformer(probe=probe, scan=scan, config=config)

In [ ]:
import torch
from usbmd.processing import rf2iq, log_compress

# Swap the 3th and 4th axes
# This is currently needed because the processing module assumes a dimension
# order that is different from the one used in the beamformers
data = np.swapaxes(data, 3, 4)

# Transform the data to IQ data
iq_data = rf2iq(data,
                fs=scan.fs,
                fc=scan.fc,
                bandwidth=probe.bandwidth,
                separate_channels=False)

# Swap the 3th and 4th axes back
iq_data = np.swapaxes(iq_data, 3, 4)

# Turn the data into a torch tensor
iq_data = torch.from_numpy(iq_data)

# Beamform the data
beamformed_data = beamformer(iq_data)['beamformed']

image = beamformed_data.numpy()[0, :, :, 0]

image = np.abs(image)

image = image/image.max()
image = log_compress(image)

print(f'Image shape: {image.shape}')

## Plotting the result

In [ ]:
import matplotlib.pyplot as plt

# Plot the image
fig, ax = plt.subplots(figsize=(6, 6))
ax.imshow(image,
          cmap='gray',
          extent=[scan.xlims[0], scan.xlims[1], scan.zlims[1], scan.zlims[0]],
          vmin=config.data.dynamic_range[0],
          vmax=config.data.dynamic_range[1])

# Add labels
ax.set_xlabel('Lateral distance [m]')
ax.set_ylabel('Depth [m]')

# Turn the figure black
ax.set_facecolor((0, 0, 0))
fig.patch.set_facecolor((0, 0, 0))

# Turn the ticks white
ax.tick_params(axis='both', colors='white')

# Turn the labels white
ax.xaxis.label.set_color('white')
ax.yaxis.label.set_color('white')

Since this data file also already has processed image data inside we can plot this too and see if the result is the same.

In [ ]:
from usbmd.processing import log_compress

try:
    with h5py.File(data_path, 'r') as f:
        image = f['data']['beamformed_data'][0, :, :]
except KeyError:
    image = np.ones((128, 128))

# Apply log compression
image = log_compress(image/np.max(image))

fig, ax = plt.subplots(figsize=(6, 6))
ax.imshow(image,
          cmap='gray',
          extent=[scan.xlims[0], scan.xlims[1], scan.zlims[1], scan.zlims[0]],
          vmin=config.data.dynamic_range[0],
          vmax=config.data.dynamic_range[1])

# Add labels
ax.set_xlabel('Lateral distance [m]')
ax.set_ylabel('Depth [m]')

# Turn the figure black
ax.set_facecolor((0, 0, 0))
fig.patch.set_facecolor((0, 0, 0))

# Turn the ticks white
ax.tick_params(axis='both', colors='white')

# Turn the labels white
ax.xaxis.label.set_color('white')
ax.yaxis.label.set_color('white')